In [ ]:
import os
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import time
from collections import Counter

# --- Konfigurasi ---
NAMA_FOLDER_INPUT = 'input'
NAMA_FOLDER_OUTPUT_STEMMED_RELATIF = 'output/sastrawi'
NAMA_FOLDER_OUTPUT_CLEANED_RELATIF = 'output/cleaned'
# --- Akhir Konfigurasi ---

base_dir = os.getcwd()
print(f"Menggunakan direktori kerja saat ini sebagai basis: {base_dir}")
input_folder_path = os.path.join(base_dir, NAMA_FOLDER_INPUT)
output_folder_stemmed_path = os.path.join(base_dir, NAMA_FOLDER_OUTPUT_STEMMED_RELATIF)
output_folder_cleaned_path = os.path.join(base_dir, NAMA_FOLDER_OUTPUT_CLEANED_RELATIF)

# --- Fungsi untuk Membersihkan Teks (Mempertahankan Newline) ---
def bersihkan_teks_preserve_lines(teks):
    """Membersihkan teks dari tanda baca, angka, dan spasi berlebih
       di dalam baris, sambil mempertahankan newline."""

    lines = teks.splitlines() # Pecah teks menjadi baris-baris
    cleaned_lines = []

    for line in lines:
        # 1. Ubah ke lowercase
        line = line.lower()
        # 2. Hapus angka
        line = re.sub(r"\d+", "", line)
        # 3. Hapus tanda baca
        tanda_baca_escaped = re.escape(string.punctuation)
        line = re.sub(r'[' + tanda_baca_escaped + ']', '', line)
        # 4. Hapus spasi/tab berlebih DI DALAM baris menjadi satu spasi
        line = re.sub(r'[ \t]+', ' ', line)
        # 5. Hapus spasi di awal/akhir setiap baris
        line = line.strip()

        # Hanya tambahkan baris jika tidak kosong setelah dibersihkan
        # (Opsional: jika ingin mempertahankan baris kosong asli,
        # hapus kondisi 'if line:')
        if line:
            cleaned_lines.append(line)

    # Gabungkan kembali baris-baris dengan newline
    return '\n'.join(cleaned_lines)
# --- Akhir Fungsi Cleaning ---

# 1. Inisialisasi Stemmer Sastrawi
print("Menginisialisasi stemmer Sastrawi...")
factory = StemmerFactory()
stemmer = factory.create_stemmer()
print("Stemmer siap.")

# 2. Buat folder output
for folder_path in [output_folder_stemmed_path, output_folder_cleaned_path]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder output dibuat: {folder_path}")
    else:
        print(f"Folder output sudah ada: {folder_path}")

# Variabel Evaluasi
unique_words_cleaned = set()
unique_words_stemmed = set()
total_words_cleaned = 0
total_words_stemmed = 0

# 3. Proses setiap file
print(f"\nMemulai proses cleaning dan stemming dari folder: {input_folder_path}")
start_time = time.time()

try:
    if not os.path.isdir(input_folder_path):
        raise FileNotFoundError(f"Folder input '{input_folder_path}' tidak ditemukan.")

    list_file = os.listdir(input_folder_path)
    processed_files = 0
    skipped_files = 0

    if not list_file:
        print("Folder input kosong.")

    for filename in list_file:
        input_file_path = os.path.join(input_folder_path, filename)

        if os.path.isfile(input_file_path) and filename.lower().endswith('.txt'):
            output_file_stemmed_path = os.path.join(output_folder_stemmed_path, filename)
            output_file_cleaned_path = os.path.join(output_folder_cleaned_path, filename)

            print(f"  -> Memproses file: {filename}...")

            try:
                with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as f_in:
                    original_text = f_in.read()

                # *** Gunakan Fungsi Cleaning yang Baru ***
                cleaned_text = bersihkan_teks_preserve_lines(original_text)

                # Simpan Teks yang Hanya Dibersihkan
                with open(output_file_cleaned_path, 'w', encoding='utf-8') as f_clean:
                    f_clean.write(cleaned_text)

                # Lakukan stemming (Sastrawi bisa handle string multi-baris)
                stemmed_text = stemmer.stem(cleaned_text)

                # Tulis hasil stemming
                with open(output_file_stemmed_path, 'w', encoding='utf-8') as f_stemmed:
                    f_stemmed.write(stemmed_text)

                # Update data evaluasi (split akan menangani multi-baris)
                # Note: Ini akan menghitung kata di semua baris seolah satu dokumen
                cleaned_words = cleaned_text.split()
                stemmed_words = stemmed_text.split()

                unique_words_cleaned.update(cleaned_words)
                unique_words_stemmed.update(stemmed_words)
                total_words_cleaned += len(cleaned_words)
                total_words_stemmed += len(stemmed_words)

                processed_files += 1

            except Exception as e:
                print(f"     ERROR saat memproses file {filename}: {e}")
        else:
            if os.path.isfile(input_file_path):
                 print(f"  -> Melewati file non-txt: {filename}")
            else:
                 print(f"  -> Melewati item yang bukan file: {filename}")
            skipped_files += 1

    end_time = time.time()
    total_time = end_time - start_time

    print(f"\nProses cleaning dan stemming selesai.")
    print(f"Jumlah file .txt yang diproses: {processed_files}")
    if skipped_files > 0:
        print(f"Jumlah item non-txt/subfolder yang dilewati: {skipped_files}")
    print(f"Hasil stemming disimpan di: {output_folder_stemmed_path}")
    print(f"Hasil cleaning (tanpa stemming) disimpan di: {output_folder_cleaned_path}")
    print(f"Total waktu eksekusi: {total_time:.2f} detik")

    # Cetak Hasil Evaluasi Reduksi Kosakata
    print("\n--- Evaluasi Reduksi Kosakata ---")
    count_unique_cleaned = len(unique_words_cleaned)
    count_unique_stemmed = len(unique_words_stemmed)
    print(f"Total kata (tokens) setelah cleaning: {total_words_cleaned}")
    print(f"Total kata (tokens) setelah stemming: {total_words_stemmed}")
    print(f"Jumlah kata unik (types) setelah cleaning: {count_unique_cleaned}")
    print(f"Jumlah kata unik (types) setelah stemming: {count_unique_stemmed}")

    if count_unique_cleaned > 0:
        reduction_percentage = ((count_unique_cleaned - count_unique_stemmed) / count_unique_cleaned) * 100
        print(f"Persentase reduksi kosakata unik: {reduction_percentage:.2f}%")
    else:
        print("Tidak ada kata unik untuk menghitung reduksi.")

except FileNotFoundError as fnf_error:
    print(f"ERROR: {fnf_error}")
    print("Pastikan folder input ada.")
except Exception as e:
    print(f"Terjadi kesalahan umum: {e}")

Menggunakan direktori kerja saat ini sebagai basis: /home/xerces/project/stemming-project
Menginisialisasi stemmer Sastrawi...
Stemmer siap.
Folder output sudah ada: /home/xerces/project/stemming-project/output/sastrawi
Folder output dibuat: /home/xerces/project/stemming-project/output/cleaned

Memulai proses cleaning dan stemming dari folder: /home/xerces/project/stemming-project/input/indo
  -> Memproses file: Kel3_Peran Bimbingan dan Konseling Dalam Pendidikan Karakter    .txt...
  -> Memproses file: Kel6_Benarkah anak-anak butuh mata pelajaran koding dan AI di sekolah.txt...
  -> Memproses file: Dampak Tarif Resiprokal Trump terhadap Industri di Indonesia_1.txt...
  -> Melewati file non-txt: Kelompok 8_Ketika Jas Putih Menjadi Tameng Menggugat Sistem Pendidikan Dokter Spesialis di Indonesia.docx
  -> Memproses file: Global South dan Ilusi Netralitas_10.txt...
  -> Memproses file: Eksistensi Media Massa Nasional_5.txt...
  -> Memproses file: Peran Media Massa dalam Membentuk Opini P